# <center> Panongbene Jean Mohamed Sawadogo </center>#

* $\color{red}{Course}$        : Visual Analytics, CentraleSupelec
* $\color{red}{Professor}$     : Petra Isenberg, Wesley Willett
* $\color{red}{Assignement}$   : second Assignement
* $\color{red}{Author}$        : Panongbene Jean Mohamed Sawadogo
* $\color{red}{Title}$         : Collecting & Cleaning Data

$\color{red}{Project - description : }$  In this project, we are scraping the site https://www.populationdata.net/pays/ to extract all informations relating to the countries of the world at the end, we will build a csv containing the countries with as much information as possible about each country.





# We import the necessary libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 

# <center> Part 1 : Scraping Data From the Web </center> #
In this part, we build and declare all the functions and variables needed to retrieve data from the website.

# All Extras Parameters

In [2]:
link_web_page = 'https://www.populationdata.net/pays/'

# the real name in the web page of the dataframe columns
array_column = ['Nom officiel','Nom propre','Continent','Sous-continent','Population',
                'Croissance démographique','Superficie','Densité','PIB','PIB/habitant',
                'Croissance du PIB','Espérance de vie',
                'Taux de natalité','Indice de fécondité','Taux de mortalité','Taux de mortalité infantile(‰ (2019))',
                'Taux d\'alphabétisation','Langues officielles','Monnaie','IDH','IPE','Nature de l"État','Chef de l"État',
                'Fête nationale','Codes ISO','Gentilé','Touristes']

# the names of all datafram columns
datafram_column = ['Nom officiel','Nom propre','Continent','Sous-continent','Population(habitants (2020))',
                  'Croissance démographique(% / an)','Superficie(km²)','Densité(habitants / km²)',
                  'PIB(milliards $USD (2017))','PIB/habitant','Croissance du PIB(% / an (2018))',
                  'Espérance de vie(ans (2019))','Taux de natalité(‰ (2019))','Indice de fécondité(enfants / femme (2019))',
                  'Taux de mortalité(‰ (2019))','Taux de mortalité infantile(‰ (2019))','Taux d"alphabétisation(% (2020))',
                   'Langues officielles','Monnaie','IDH(/ 1 (2018))','IPE','Nature de l"État','Chef de l"État','Fête nationale',
                  'Codes ISO','Gentilé','Touristes']

In [3]:
def add_new_line_data_fram(name, dictionnary_data_new,datafram_country_data):
    """Use this function to add a new row in our data frame. The new row values ​​are
    the values ​​in dictionnary_data_new"""
    
    # We create the new line in data Fram
    datafram_country_data.loc[name] = 'NAN'
    
    # We put the values in our new line
    j = 0
    for i in dictionnary_data_new:
        datafram_country_data.loc[name][j] = dictionnary_data_new[i]
        j+=1

def save_data_frame_csv(file_name, dataframe_data):
    """We use this function to save dataframe in the csv file"""
    my_file_name = file_name + '.csv'
    dataframe_data.to_csv(my_file_name)
    print("Saved " +my_file_name)

## Function to Scrape and analyze data


In [4]:
#function 
def fetch_scrapping_pages():
    """We use this function to scrapes all informations about each country. 
     We use also this function for cleaning the data in the datafram. At the 
     end this function return a panda dataframe which contains all cleaning informations 
     of each countries. """
    
    # page Request
    page_country_first = requests.get(link_web_page, allow_redirects=True)
    
    #we process the request
    if not page_country_first.status_code == 200:
        print(" ERROR Status code: " + str(page_country_first.status_code))
        return
    else:
        soup = BeautifulSoup(page_country_first.content)
        
        # we create the data frame
        datafram_country_data = pd.DataFrame(columns=datafram_column)
    
        for tablecountry in soup.findAll('table'):
            for onecountry in tablecountry.findAll('tr'):
                # name country
                name_country = ''
                # we create the dictionnary
                dictionnary_data = {}
                
                if('data-iso-code' in onecountry.attrs):
                    for country in onecountry.findAll('a'):
                        if('pays' in country['href']):
                            link_country = country['href']
                            name_country = country.text
                            
                            # page Request
                            page_country = requests.get(link_country, allow_redirects=True)
                            #we process the request
                            if not page_country.status_code == 200:
                                print(" ERROR Status code: " + str(page_country.status_code))
                                continue
                            else:
                                soup_page_country = BeautifulSoup(page_country.content)
                                
                                for tablecountry in soup_page_country.findAll('tbody'):
                                    for table_infos in tablecountry.findAll('tr'):
                                        keys_values = []
                                        for information_line in table_infos.findAll('td'):
                                            keys_values.append(information_line.text)
                                       
                                        #We remove all values ; because ; is interpreted by csv as a column separator
                                        keys_values[0] = keys_values[0].replace(";","")
                                        keys_values[1] = keys_values[1].replace(";","")
                                        
                                        #we replace all nc value by NaN
                                        keys_values[0] = keys_values[0].replace("n.c.","NaN")
                                        keys_values[1] = keys_values[1].replace("n.c.","NaN")

                                        dictionnary_data[keys_values[0]] = keys_values[1]
                                    #we use break here because some page have a lot of tbody
                                    break
                                # we add the new ligne in the datafram
                                add_new_line_data_fram(name_country,dictionnary_data,datafram_country_data)
    return datafram_country_data

In [5]:
datafram_country_data = fetch_scrapping_pages()
datafram_country_data.head()

,Nom officiel,Nom propre,Continent,Sous-continent,Population(habitants (2020)),Croissance démographique(% / an),Superficie(km²),Densité(habitants / km²),PIB(milliards $USD (2017)),PIB/habitant,...,Langues officielles,Monnaie,IDH(/ 1 (2018)),IPE,"Nature de l""État","Chef de l""État",Fête nationale,Codes ISO,Gentilé,Touristes
Abkhazie,République d'Abkhazie,Аҧсны Аҳәынҭқарра (ab) აფხაზეთი (ka) Республ...,Asie,Asie de l'Ouest,246 313 habitants (2018),"0,49 % / an",8 653 km²,"28,47 habitants / km²","0,336 milliards $USD (2017)",1 372 $USD (2017),...,"Abkhaze, géorgien, russe","Apsar abkhaze, Rouble russe (₽ RUB)",NaN,NaN,République,Président Raul Khadjimba,23 juillet (indépendance de 1992),",","Abkhaze, Abkhazienne, Abkhazien",1 000 000 personnes (2017)
Açores,Région autonome des Açores,Região Autónoma dos Açores (pt),Europe,Union Européenne,243 862 habitants (2018),"-0,39 % / an",2 322 km²,"105,02 habitants / km²","4,220 milliards $USD (2015)",17 398 $USD (2015),...,Portugais,Euro (€ EUR),NaN,NaN,République,Président Marcelo Rebelo de Sousa,10 juin (mort de Luís de Camões en 1580),",","Açoréenne, Açoréen, Portugaise, Portugais",1 487 428 personnes (2016)
Afghanistan,République islamique d'Afghanistan,د أفغانستان اسلامي جمهوریت (ps) جمهوری اسلامی...,Asie,Asie centrale,34 233 212 habitants (2020),"6,23 % / an",652 864 km²,"52,44 habitants / km²","19,363 milliards $USD (2018)",521 $USD (2018),...,"Dari (persan), pachtou",Afghani (AFN),"0,496 / 1 (2018)","37,74 (2018)",République islamique,Président Ashraf Ghani,19 août (indépendance de 1919),"AF, AFG",Afghan,20 000 personnes (2016)
Afrique du Sud,République d'Afrique du Sud,Republiek van Suid-Afrika (af) Republic of So...,Afrique,Afrique Australe,57 725 606 habitants (2018),"2,13 % / an",1 219 912 km²,"47,32 habitants / km²","366,298 milliards $USD (2018)",6 340 $USD (2018),...,"Zoulou, afrikaans, anglais (11 langues)",Rand (ZAR),"0,705 / 1 (2018)","44,73 (2018)","République unitaire parlementaire, régime prés...",Président Cyril Ramaphosa,27 avril (premières élections multiraciales en...,"ZA, ZAF","Sud-Africaine, Sud-Africain",10 285 000 personnes (2017)
Albanie,République d'Albanie,Republika e Shqipërisë (sq),Europe,Europe (hors Union Européenne),2 870 324 habitants (2018),"-0,22 % / an",28 748 km²,"99,84 habitants / km²","15,059 milliards $USD (2018)",5 254 $USD (2018),...,Albanais,Lek (ALL),"0,791 / 1 (2018)","65,46 (2018)",République,Président Ilir Meta,28 novembre (indépendance de 1912),"AL, ALB","Albanaise, Albanais",5 375 174 personnes (2017)


In [6]:
# we save the data int the local repertory
save_data_frame_csv('Lab2-country_Information_scrapping', datafram_country_data)

Saved Lab2-country_Information_scrapping.csv


# <center> Part 2 : Cleaining data </center> #
In this part, we explain how we use Openrefine for cleaning data

<span style="color:red"><font size="+1"> INTRODUCTION : </font></span> After scraping the data and saving it to a csv file, we will clean the data using OpenRefine.
We must clean the data because if we take for example the population column, we expect to have a numerical value but, we end up with values of this type <span style="color:green">57 725 606 habitants (2018)</span>.
In this part, we'll see how we use OpenRefine to clean up data.

<span style="color:red"><font size="+1"> Step 1 Install and run openRefine : </font></span> We download, unzip openrif software in this website  https://openrefine.org/download.html. We run OpenRefine binary program with this command <span style="color:green">./refine </span>
<img src="imageExplainOpenRefine/1.png"/>
<img src="imageExplainOpenRefine/2.png"/>
After we run openrefine, the software open a webserver in a local host using the port 3333 <span style="color:green">(127.0.0.1:3333)</span>


<span style="color:red"><font size="+1"> Step 2 Load the Data : </font></span> We load the csv file we have that we built in part one and we create a new project.
<img src="imageExplainOpenRefine/3.png"/>


<span style="color:red"><font size="+1"> Step 3 Cleaning the population column : </font></span> We divided the population column into two columns, the first column contains the population number and the second column contains the date of the census
<img src="imageExplainOpenRefine/4.png"/>
Before cleaning the column

<img src="imageExplainOpenRefine/5.png"/>
After cleaning the population column



<span style="color:red"><font size="+1"> Step 4 Cleaning the Croissance démographique, Superficie and Densité columns : </font></span> 
We transformed the values of the Croissance démographique, Superficie and Densité columns into numerical values
    
<img src="imageExplainOpenRefine/6.png"/>
Before cleaning the column

<img src="imageExplainOpenRefine/7.png"/>
After cleaning the population column


<span style="color:red"><font size="+1"> Step 5 Clean the last columns : </font></span> 
For cleaning up the remaining columns, for some we have divided each column into two columns: the first contains the numeric value of the column and the second the date corresponding to that value. We've also changed the names of some columns to make them more meaningful.


<img src="imageExplainOpenRefine/8.png"/>
Before cleaning the column

<img src="imageExplainOpenRefine/9.png"/>
After cleaning the population column



<span style="color:red"><font size="+1"> Step 6 Finish the job : </font></span> For this step, we replaced the empty cells with NaN and we managed the cells that did not pass the filters individually.

# The head of the file Result of OpenRefine Cleaning

In [7]:
#data_openrefine_result = pd.read_csv("Lab2-country_Information_scrapping-result-openrefine-operation.xlsx")
data_openrefine_result = pd.read_excel("Lab2-country_Information_scrapping-result-openrefine-operation.xlsx")

#pd.read_excel("the_document.ods", engine="odf")

In [8]:
data_openrefine_result

,Column,Nom officiel,Nom propre,Continent,Sous-continent,Population(habitants),Date du recensement de la population,Croissance démographique(% / an),Superficie(km²),Densité(habitants / km²),...,IDH(/ 1 (2018)) 2,IPE,date recensement IPE,"Nature de l""État","Chef de l""État",Fête nationale,Codes ISO,Gentilé,Touristes,date recensement Touristes
0,Abkhazie,République d'Abkhazie,Аҧсны Аҳәынҭқарра (ab) აფხაზეთი (ka) Республ...,Asie,Asie de l'Ouest,246 313,2018,"0,49",8 653,"28,47",...,NaN,NaN,NaN,République,Président Raul Khadjimba,23 juillet (indépendance de 1992),",","Abkhaze, Abkhazienne, Abkhazien",1 000 000,2017.0
1,Açores,Région autonome des Açores,Região Autónoma dos Açores (pt),Europe,Union Européenne,243 862,2018,"-0,39",2 322,"105,02",...,NaN,NaN,NaN,République,Président Marcelo Rebelo de Sousa,10 juin (mort de Luís de Camões en 1580),",","Açoréenne, Açoréen, Portugaise, Portugais",1 487 428,2016.0
2,Afghanistan,République islamique d'Afghanistan,د أفغانستان اسلامي جمهوریت (ps) جمهوری اسلامی...,Asie,Asie centrale,34 233 212,2020,"6,23",652 864,"52,44",...,2018.0,"37,74",2018.0,République islamique,Président Ashraf Ghani,19 août (indépendance de 1919),"AF, AFG",Afghan,20 000,2016.0
3,Afrique du Sud,République d'Afrique du Sud,Republiek van Suid-Afrika (af) Republic of So...,Afrique,Afrique Australe,57 725 606,2018,"2,13",1 219 912,"47,32",...,2018.0,"44,73",2018.0,"République unitaire parlementaire, régime prés...",Président Cyril Ramaphosa,27 avril (premières élections multiraciales en...,"ZA, ZAF","Sud-Africaine, Sud-Africain",10 285 000,2017.0
4,Albanie,République d'Albanie,Republika e Shqipërisë (sq),Europe,Europe (hors Union Européenne),2 870 324,2018,"-0,22",28 748,"99,84",...,2018.0,"65,46",2018.0,République,Président Ilir Meta,28 novembre (indépendance de 1912),"AL, ALB","Albanaise, Albanais",5 375 174,2017.0
5,Algérie,République algérienne démocratique et populaire,الجمهورية الجزائرية الديمقراطية الشعبية (ar),Afrique,Afrique du Nord,43 820 839,2020,"1,90",2 381 741,"18,40",...,2018.0,"57,18",2018.0,République,Président Abdelmadjid Tebboune,1er novembre (déclenchement de la révolution e...,"DZ, DZA","Algérienne, Algérien",2 451 000,2017.0
6,Allemagne,République Fédérale d'Allemagne,Bundesrepublik Deutschland (de),Europe,Union Européenne,83 042 235,2019,"0,30",357 386,"232,36",...,2018.0,"78,37",2018.0,République constitutionnelle parlementaire féd...,Président Frank-Walter Steinmeier Chancelière...,3 octobre (Réunification de 1990),"DE, DEU","Allemande, Allemand",37 452 000,2017.0
7,Andorre,Principauté d'Andorre,Principat d'Andorra (ca),Europe,Europe (hors Union Européenne),76 522,2018,"2,31",468,"163,51",...,2018.0,NaN,NaN,Coprincipauté parlementaire,Coprince français Emmanuel Macron (président f...,8 septembre,"AD, AND","Andorranne, Andorran",2 999 722,2017.0
8,Angola,République d'Angola,República de Angola (pt) Repubilika ya Ngola ...,Afrique,Afrique Australe,29 131 016,2018,"2,72",1 281 432,"22,73",...,2018.0,"37,44",2018.0,"République présidentielle, régime autoritaire",Président João Lourenço,11 novembre (indépendance de 1975),"AO, AGO","Angolaise, Angolais",397 000,2016.0
9,Anguilla,Anguilla,Anguilla,Amériques,Antilles,15 174,2019,"0,86",91,"166,75",...,2008.0,NaN,NaN,Territoire britannique d'outre-mer,Reine Élisabeth II Gouverneur Tim Foy,30 mai,"AI, AIA","Anguillaise, Anguillais",68 000,2017.0


# Json file result of OpenRefine Cleaning

In [9]:
openrefine_history_result = pd.read_json("Lab2-historyOpenRefineScrapping.json")

In [10]:
openrefine_history_result

,columnName,description,edits,engineConfig,expression,guessCellType,maxColumns,mode,newColumnName,oldColumnName,onError,op,regex,removeOriginalColumn,repeat,repeatCount,separator
0,Population(habitants (2020)),Split column Population(habitants (2020)) by s...,NaN,"{'facets': [], 'mode': 'row-based'}",NaN,1.0,0.0,separator,NaN,NaN,NaN,core/column-split,0.0,1.0,NaN,NaN,habitants
1,Population(habitants (2020)) 2,Text transform on cells in column Population(h...,NaN,"{'facets': [], 'mode': 'row-based'}","grel:value.replace(""("","""").replace("")"","""")",NaN,NaN,NaN,NaN,NaN,keep-original,core/text-transform,NaN,NaN,0.0,10.0,NaN
2,NaN,Rename column Population(habitants (2020)) 2 t...,NaN,NaN,NaN,NaN,NaN,NaN,Date du recensement de la population,Population(habitants (2020)) 2,NaN,core/column-rename,NaN,NaN,NaN,NaN,NaN
3,Croissance démographique(% / an),Text transform on cells in column Croissance d...,NaN,"{'facets': [], 'mode': 'row-based'}","grel:value.replace("" % / an"","""")",NaN,NaN,NaN,NaN,NaN,keep-original,core/text-transform,NaN,NaN,0.0,10.0,NaN
4,Superficie(km²),Text transform on cells in column Superficie(k...,NaN,"{'facets': [], 'mode': 'row-based'}","grel:value.replace("" km²"","""")",NaN,NaN,NaN,NaN,NaN,keep-original,core/text-transform,NaN,NaN,0.0,10.0,NaN
5,Densité(habitants / km²),Text transform on cells in column Densité(habi...,NaN,"{'facets': [], 'mode': 'row-based'}","grel:value.replace("" habitants / km²"","""")",NaN,NaN,NaN,NaN,NaN,keep-original,core/text-transform,NaN,NaN,0.0,10.0,NaN
6,PIB(milliards $USD (2017)),Split column PIB(milliards $USD (2017)) by sep...,NaN,"{'facets': [], 'mode': 'row-based'}",NaN,1.0,0.0,separator,NaN,NaN,NaN,core/column-split,0.0,1.0,NaN,NaN,milliards $USD
7,PIB(milliards $USD (2017)) 2,Text transform on cells in column PIB(milliard...,NaN,"{'facets': [], 'mode': 'row-based'}","grel:value.replace(""("","""").replace("")"","""")",NaN,NaN,NaN,NaN,NaN,keep-original,core/text-transform,NaN,NaN,0.0,10.0,NaN
8,PIB/habitant,Split column PIB/habitant by separator,NaN,"{'facets': [], 'mode': 'row-based'}",NaN,1.0,0.0,separator,NaN,NaN,NaN,core/column-split,0.0,1.0,NaN,NaN,$USD
9,PIB/habitant 2,Text transform on cells in column PIB/habitant...,NaN,"{'facets': [], 'mode': 'row-based'}","grel:value.replace(""("","""").replace("")"","""")",NaN,NaN,NaN,NaN,NaN,keep-original,core/text-transform,NaN,NaN,0.0,10.0,NaN
